In [2]:
#importing libraries, including langchain libraries
#langchain explanation:
#LangChain is a high-level framework designed to simplify interaction with LLMs by abstracting prompts,
#memory, tool usage, and chaining steps together. It’s especially useful for building LLM-powered applications.

import os
import pandas as pd
from dotenv import load_dotenv

# PromptTemplate allows us to define and manage prompt strings dynamically with variable placeholders.
# This is better than manually formatting strings.
from langchain.prompts import PromptTemplate

#this import is used to split text into chunks to improve natural language processing
from langchain.text_splitter import RecursiveCharacterTextSplitter

# This is a LangChain-compatible wrapper that allows calling Groq-hosted LLMs - we used Mistral
from langchain_groq import ChatGroq

from tqdm import tqdm
import json
import re

In [3]:
#here we are loading our environment variables - we generated an API Key by creating an account on Groq
#we will not upload this to Git to maintain privacy and misuse

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [4]:
#using pandas dataframe to load the parquet file from our previous step
df = pd.read_parquet(r"C:\Users\kusha\Desktop\financial-risk-analyzer\data\10K_extracted_sections.parquet")

In [5]:
#creating a list of 5 set risk categories that we want our LLM to classify
narrative_list = [
    "Legal & Regulatory",
    "Financial Distress",
    "Supply Chain",
    "Executive Turnover",
    "Market Dependency"
]

In [6]:
#here we are creating a prompt template that will be used by our LLM model
#this essentially are instructions to our model telling it what it should be doing
#here we add explicit conditions to ensure that the output only contains the json format and not any other explanation
#this ensures the next steps of our code work correctly, and receive expected input

prompt_template = PromptTemplate.from_template("""
You are a financial risk analyst.

Given the following 10-K sections:
- Item 1A (Risk Factors)
- Item 7 (Management Discussion & Analysis)

Your task is to:
1. Classify the overall risk into ONE of the following categories:
{narratives}

2. Assign a risk score from 1 to 5, where 5 is the highest risk.

Only return valid JSON. Do not include explanations, reasoning, or any non-JSON text.

JSON format:
{{
  "narrative_category": "...",
  "risk_score": 3
}}

Text:
{content}
""")

In [7]:
# Combining our extracted Item 1A and Item 7 sections into a single string for input to the LLM

def chunk_text(text1, text2):
    combined = (text1 or "") + "\n\n" + (text2 or "")
    return [combined[:4000]]  # we're splitting our text to maximum 4k characters since our LLM has a limit of 8000

#this essentially just sends our data in chunks to the LLM

In [8]:
#here we are initializing our model with no randomness, to ensure reproducibility
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [9]:
#here we are defining a function that will extract information from our json response
#if we dont find any response, it will respond with an error
def extract_json_from_response(text):
    match = re.search(r"```(?:json)?\s*(\{.*?\})\s*```", text, re.DOTALL)
    if match:
        return json.loads(match.group(1))
    match = re.search(r"(\{.*\})", text, re.DOTALL)
    if match:
        return json.loads(match.group(1))
    raise ValueError("No valid JSON found in response")


In [10]:
#here we are filtering out rows and keeping values where either 1A or 7 or both items are present
#to put togeher this logic, we use OR (|)

df_filtered = df[
    (df['item_1a_text'].notnull() & df['item_1a_text'].str.strip().ne("")) |
    (df['item_7_text'].notnull() & df['item_7_text'].str.strip().ne(""))
]

print(f"✅ Total rows with valid Item 1A or 7: {len(df_filtered)}")

#we now convert it into a subset
df_subset = df_filtered


#initializing our output
outputs = []

# we now loop through each value in our subset
for idx, row in tqdm(df_subset.iterrows(), total=len(df_subset)):
    try:
        ticker = row.get("ticker", "N/A")
        year = row.get("year", "N/A")
        item_1a = (row.get("item_1a_text") or "").strip()
        item_7 = (row.get("item_7_text") or "").strip()

        #here we do a redundancy check and raise an error if we find any rows with both items missing
        if not item_1a and not item_7:
            raise ValueError("Both Item 1A and 7 are empty")

        print(f"\n🔍 Processing row {idx} | Ticker: {ticker}, Year: {year}")

        # here we use our chunks function that we defined above and join both items
        chunks = chunk_text(item_1a, item_7)
        print(f"🔹 {len(chunks)} chunk(s) generated")

        responses = []

        #now we feed our chunks to LLM and use the prompt we created above
        for i, chunk in enumerate(chunks):
            prompt = prompt_template.format(
                narratives=narrative_list,
                content=chunk
            )

            print(f"🧠 Invoking LLM on chunk {i+1}/{len(chunks)}...")
            response = llm.predict(prompt)

            #if we dont receive a response, we return an error
            if not response.strip():
                raise ValueError("LLM returned empty response")

            #we now parse the json we received from our response above and add errors below
            try:
                parsed = extract_json_from_response(response)
            except json.JSONDecodeError:
                print(f"❌ JSON parsing failed for chunk {i+1}: {response}")
                raise
            except ValueError as ve:
                print(f"❌ ValueError in chunk {i+1}: {ve}")
                raise

            #if our response does not include both values in json, we raise an error indicating the LLM has provided incorrect response
            if not all(k in parsed for k in ("narrative_category", "risk_score")):
                raise ValueError(f"Missing keys in parsed response: {parsed}")

            responses.append(parsed)

        #due to limitations in API calls to our LLM, we use one chunk per row
        r = responses[0]
        outputs.append({
            "ticker": ticker,
            "year": year,
            "item1a": item_1a,
            "item7": item_7,
            "narrative": r["narrative_category"],
            "risk_score": r["risk_score"]
        })

        print(f"✅ Completed row {idx} | Risk Score: {r['risk_score']}, Narrative: {r['narrative_category']}")

    except Exception as e:
        print(f"❌ Error at row {idx} ({row.get('ticker', 'N/A')}): {e}")

✅ Total rows with valid Item 1A or 7: 970


  0%|          | 0/970 [00:00<?, ?it/s]


🔍 Processing row 0 | Ticker: A, Year: 2024
🔹 1 chunk(s) generated
🧠 Invoking LLM on chunk 1/1...


C:\Users\kusha\AppData\Local\Temp\ipykernel_23048\3080012028.py:47: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(prompt)
  0%|          | 1/970 [00:00<14:43,  1.10it/s]

✅ Completed row 0 | Risk Score: 4, Narrative: Legal & Regulatory

🔍 Processing row 2 | Ticker: AAL, Year: 2025
🔹 1 chunk(s) generated
🧠 Invoking LLM on chunk 1/1...


...


🔍 Processing row 1485 | Ticker: YOU, Year: 2025
🔹 1 chunk(s) generated
🧠 Invoking LLM on chunk 1/1...
❌ Error at row 1485 (YOU): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jteba7n9ehjbkb6dypw1vck0` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500036, Requested 1145. Please try again in 3m24.103s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

🔍 Processing row 1487 | Ticker: ZBH, Year: 2025
🔹 1 chunk(s) generated
🧠 Invoking LLM on chunk 1/1...
❌ Error at row 1487 (ZBH): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jteba7n9ehjbkb6dypw1vck0` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500035, Requested 1145. Please try again in 3m24.022s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/setti

100%|█████████▉| 969/970 [1:34:57<00:00,  3.85it/s]

❌ Error at row 1490 (ZION): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jteba7n9ehjbkb6dypw1vck0` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500035, Requested 1145. Please try again in 3m23.950999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

🔍 Processing row 1491 | Ticker: ZTS, Year: 2025
🔹 1 chunk(s) generated
🧠 Invoking LLM on chunk 1/1...
❌ Error at row 1491 (ZTS): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jteba7n9ehjbkb6dypw1vck0` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500034, Requested 1145. Please try again in 3m23.878s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

🔍 Processing row 1492 | Ticker:

100%|██████████| 970/970 [1:34:57<00:00,  5.87s/it]

❌ Error at row 1492 (ZWS): Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jteba7n9ehjbkb6dypw1vck0` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500033, Requested 1145. Please try again in 3m23.648s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [11]:
# ✅ Step 5: Save to risk_matrix.parquet
df_output = pd.DataFrame(outputs)
df_output.to_parquet("risk_matrix.parquet", index=False)
print("✅ Saved results to risk_matrix.parquet")

✅ Saved results to risk_matrix.parquet


In [23]:
df_output.count()

ticker        516
year          516
item1a        516
item7         516
narrative     516
risk_score    516
dtype: int64

In [ ]:
# Our model stopped generating responses at row 606 as we ran out of tokens
# Out of 606 rows passed to the model, we successfully generated outputs for 516 filings.
# We also invoked the LLM on one combined chunk per row to reduce token usage and avoid multi-chunk overhead.
# Some rows failed due to internal service errors from the Groq API
# The final results were saved in 'risk_matrix.parquet', containing 516 rows of structured risk annotations.
# We chose Parquet format because it is more efficient than CSV — it loads faster and consumes less disk space.
# Due to financial constraints we used free API,
# however, the drawbacks include limited tokens, which forced us to use one chunk per row, and also halted our generation mid-way
# drawbacks of this could include inaccurate risk assignment, limited datasets
# this lesson will be stored and used during future improvement strategies